In [50]:
from pyspark import RDD, SparkContext

import pandas as pd
import json
import pickle

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time

In [74]:
#test_data = "../resource/asnlib/publicdata/train_review.json"
#user_data = '../resource/asnlib/publicdata/user.json'
#business_data = '../resource/asnlib/publicdata/business.json'

test_data = "yelp_val.csv"
user_data =  "data/user.json"
business_data = 'data/business.json'

output_file = "out.csv"

In [12]:
def feature_select_user(partition):
    for row in partition:
        data = json.loads(row)
        yield data['user_id'], data['useful'], data['average_stars'], data['review_count']


def feature_select_business(partition):
    for row in partition:
        data = json.loads(row)
        yield data['business_id'], float(data['stars']), float(data['review_count'])

def feature_select_train(partition):
    for row in partition:
        data = json.loads(row)
        yield data['user_id'], data['business_id'], float(0)

In [13]:
sc = SparkContext("local", "HW3").getOrCreate()
sc.setLogLevel("ERROR")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=HW3, master=local) created by __init__ at <ipython-input-4-1eb86f7b012c>:1 

In [14]:
reviews_RDD = sc.textFile(test_data).map(lambda line: line.split(','))
header = reviews_RDD.first()
reviews_RDD = reviews_RDD.filter(lambda data: data != header).map(lambda x: (x[0], x[1], float(0)))
#test_data = reviews_RDD.filter(lambda data: data != header).map(lambda x: (x[0], x[1], float(0)))

In [15]:
biz_RDD = sc.textFile(business_data).mapPartitions(feature_select_business).map(lambda x:(x[0],x))

In [16]:
user_RDD = sc.textFile(user_data).mapPartitions(feature_select_user).map(lambda x:(x[0],x))

In [17]:
rev_w_biz_RDD = reviews_RDD.map(lambda x: (x[1], (x[0],x[2]))).leftOuterJoin(biz_RDD).map(lambda x: (x[1][0][0],(list([x[1][0][1]])+list(x[1][1]))))
rev_w_biz_plus_user_RDD = rev_w_biz_RDD.leftOuterJoin(user_RDD).map(lambda x: (list([x[0]]) +  list([x[1][0][1]])+ x[1][0][2:] + list(x[1][1][1:]) + list([x[1][0][0]])))

In [18]:
main_data_file = rev_w_biz_plus_user_RDD.collect()

In [59]:
# Using Pandas only for feeding into XGB
data_df = pd.DataFrame(main_data_file)

In [60]:
train_df = data_df.iloc[: , 2:-1 ]
test_df = data_df.iloc[: , -1: ]

In [61]:
with open('model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)


In [84]:
xgb_preds = xgb_model.predict(train_df)

In [87]:
xgb_preds = pd.DataFrame(xgb_preds, columns = ['prediction'])

In [88]:
test = data_df.iloc[: , :2 ].rename(columns={0: "user_id", 1: "business_id"})

In [89]:
predictions = pd.concat([test,xgb_preds], axis=1)

In [92]:
predictions.to_csv(output_file, index=False)

In [49]:
rmse_predictions = predictions.values.tolist()

gt = sc.textFile(test_data)
csvHeader = gt.first()
gt = gt.filter(lambda x: x != csvHeader).map(lambda x: x.split(",")).map(lambda x: (x[0]+x[1],float(x[2]))).collectAsMap()


rmse = 0
skipped = 0
for p in rmse_predictions:
    try:
        rmse += (float(p[2])-gt[p[0]+p[1]])**2
    except:
        print(p)
        skipped+=1

print(skipped)
print((rmse/len(predictions))**0.5)

0
0.9841449694511238
